In [1]:
import pandas as pd
df = pd.read_json("data/COCO/coco-easier.txt", 
                  lines=True)
df = df.sample(frac=1).reset_index(drop=True)
df[:5]



,all_categories,captions,category,file_name,img_id
0,"[tennis racket, person, person, person, person...",[A tennis player with his leg up in the air an...,tennis racket,000000121744.jpg,121744
1,"[person, skis]","[A woman slashing down a snowy hill on skis., ...",skis,000000493442.jpg,493442
2,"[person, person, tennis racket]",[A man standing on a tennis court holding a ra...,tennis racket,000000289059.jpg,289059
3,"[giraffe, giraffe]",[Pair of giraffes walking on grassy area in en...,giraffe,000000150726.jpg,150726
4,[clock],[A very tall church with a clock below a tower...,clock,000000246522.jpg,246522


In [2]:
inception_df = pd.read_json("data/inception_output.txt", lines=True)
inception_df.set_index('image',inplace=True)
inception_df[:5]

,vector
image,
000000173830.jpg,"[0.27057743072509705, 0.11477771401405301, 0.0..."
000000570664.jpg,"[0.252127289772033, 0.434070438146591, 0.22712..."
000000304396.jpg,"[0.24179434776306102, 0.08653920888900701, 0.1..."
000000441491.jpg,"[0.11056154966354301, 0.08768206834793, 0.0857..."
000000289059.jpg,"[0.23210205137729603, 0.1578049659729, 0.27599..."


In [3]:
import numpy as np

def concatenate(vector1, vector2):
    a = np.array(vector1)
    b = np.array(vector2)
    return np.concatenate((a, b))

In [4]:
import doc2vec_wrapped
doc2vec_vectors_series = doc2vec_wrapped.vectorize_content()
df["concat_vectors"] = df.apply(lambda row: concatenate(doc2vec_vectors_series[row['file_name']],
                                                       inception_df['vector'][row['file_name']]), axis=1)
df[:5]

2018-05-26 19:21:43,726 : INFO : collecting all words and their counts
2018-05-26 19:21:43,728 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2018-05-26 19:21:43,763 : INFO : collected 3702 word types and 10 unique tags from a corpus of 1503 examples and 65463 words
2018-05-26 19:21:43,764 : INFO : Loading a fresh vocabulary
2018-05-26 19:21:43,779 : INFO : min_count=5 retains 1058 unique words (28% of original 3702, drops 2644)
2018-05-26 19:21:43,783 : INFO : min_count=5 leaves 61217 word corpus (93% of original 65463, drops 4246)
2018-05-26 19:21:43,793 : INFO : deleting the raw counts dictionary of 3702 items
2018-05-26 19:21:43,799 : INFO : sample=0.001 downsamples 71 most-common words
2018-05-26 19:21:43,802 : INFO : downsampling leaves estimated 39737 word corpus (64.9% of prior 61217)
2018-05-26 19:21:43,814 : INFO : estimated required memory for 1058 words and 400 dimensions: 3932600 bytes
2018-05-26 19:21:43,816 : INFO : resetting layer weight

,all_categories,captions,category,file_name,img_id,concat_vectors
0,"[tennis racket, person, person, person, person...",[A tennis player with his leg up in the air an...,tennis racket,000000121744.jpg,121744,"[0.124936044216, 0.0830611288548, 0.1788370609..."
1,"[person, skis]","[A woman slashing down a snowy hill on skis., ...",skis,000000493442.jpg,493442,"[0.199409678578, 0.0874392613769, 0.1511009037..."
2,"[person, person, tennis racket]",[A man standing on a tennis court holding a ra...,tennis racket,000000289059.jpg,289059,"[0.123697988689, 0.0936013758183, 0.1886771917..."
3,"[giraffe, giraffe]",[Pair of giraffes walking on grassy area in en...,giraffe,000000150726.jpg,150726,"[-0.0723032057285, 0.220078751445, -0.02437694..."
4,[clock],[A very tall church with a clock below a tower...,clock,000000246522.jpg,246522,"[-0.147770911455, -0.00727563071996, -0.031896..."


In [6]:
X = df["concat_vectors"]
X

0       [0.124936044216, 0.0830611288548, 0.1788370609...
1       [0.199409678578, 0.0874392613769, 0.1511009037...
2       [0.123697988689, 0.0936013758183, 0.1886771917...
3       [-0.0723032057285, 0.220078751445, -0.02437694...
4       [-0.147770911455, -0.00727563071996, -0.031896...
5       [-0.0625411197543, 0.0762711539865, -0.0753941...
6       [0.11179869622, 0.0716297030449, 0.17761084437...
7       [-0.194728344679, 0.0565872378647, 0.016416905...
8       [0.133189931512, 0.073773086071, 0.19709981977...
9       [-0.0710869282484, 0.278602838516, 0.044715769...
10      [-0.101948015392, 0.221950888634, 0.0250145085...
11      [0.18584549427, 0.115170404315, 0.116634897888...
12      [-0.0583600029349, 0.168031319976, 0.090615339...
13      [0.0681081414223, 0.0443238914013, 0.198572471...
14      [-0.121055424213, 0.080839715898, 0.0839423760...
15      [0.104296788573, 0.201710626483, 0.07316695898...
16      [0.140348821878, 0.229719787836, 0.09589786082...
17      [-0.10

In [10]:
from sklearn.neural_network import MLPClassifier
X = df["concat_vectors"].tolist()
y = df["category"].tolist()
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(5, 2), random_state=1)
clf.fit(X, y)                         
MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto',
       beta_1=0.9, beta_2=0.999, early_stopping=False,
       epsilon=1e-08, hidden_layer_sizes=(5, 2), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)



MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(5, 2), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [12]:
df.loc[df['img_id'] == 125850].concat_vectors

809    [-0.115660868585, 0.103114269674, -0.026257732...
Name: concat_vectors, dtype: object

In [16]:
clf.predict(df.loc[df['img_id'] == 125850].concat_vectors.tolist())[0]

'traffic light'